In [ ]:
#r "nuget: Microsoft.DotNet.Interactive.SqlServer, *-*"

In [ ]:
#!connect mssql --kernel-name sql2025 --connection-string "Server=localhost;TrustServerCertificate=True;Integrated Security=True"

In [ ]:
SELECT @@VERSION

In [ ]:
USE master
IF EXISTS (SELECT name FROM sys.databases WHERE name='RESTAPI')BEGIN
    ALTER DATABASE RESTAPI SET SINGLE_USER WITH ROLLBACK IMMEDIATE;
    DROP DATABASE RESTAPI;
END
GO
CREATE DATABASE RESTAPI
GO
USE RESTAPI
GO
create master key encryption by password = 'MyTest!Mast3rP4ss'
GO
sp_configure 'external rest endpoint enabled', 1;
RECONFIGURE WITH OVERRIDE

In [ ]:
DECLARE @response NVARCHAR(max)

EXEC Sp_invoke_external_rest_endpoint
  @url = N'https://datausa.io/api/data?drilldowns=State&measures=Population&year=latest',
  @method = 'GET',
  @response = @response output;

SELECT TOP 5 * FROM Openjson(Json_query(@response, '$.result.data')) A 

In [ ]:
DECLARE @response NVARCHAR(max)

EXEC Sp_invoke_external_rest_endpoint
  @url = N'https://reqres.in/api/products/',
  @headers = '{"x-api-key":"reqres-free-v1"}',
  @method = 'GET',
  @response = @response output;

SELECT * FROM   Openjson(Json_query(@response, '$.result.data')) A 

In [ ]:
CREATE DATABASE SCOPED CREDENTIAL [https://reqres.in/api/]
WITH IDENTITY = 'HTTPEndpointHeaders', SECRET = '{"x-api-key":"reqres-free-v1"}';


In [ ]:
DECLARE @response NVARCHAR(max)

EXEC Sp_invoke_external_rest_endpoint
  @url = N'https://reqres.in/api/products/',
  @credential = [https://reqres.in/api/],
  @method = 'GET',
  @response = @response output;

SELECT * FROM   Openjson(Json_query(@response, '$.result.data')) A 

In [ ]:
DECLARE @response NVARCHAR(max)

EXEC Sp_invoke_external_rest_endpoint
  @url = N'https://reqres.in/api/products/',
  @credential = [https://reqres.in/api/],
  @method = 'GET',
  @response = @response output;

SELECT Json_value(value, '$.name')          name,
       Json_value(value, '$.color')         color,
       Json_value(value, '$.pantone_value') pantone_value
FROM   Openjson(Json_query(@response, '$.result.data'), 'strict $') AS export 

In [ ]:
DECLARE @response NVARCHAR(max)
DECLARE @prompt NVARCHAR(max) = N'what is the airspeed velocity of an unladen swallow?'
DECLARE @model NVARCHAR(250) = N'smollm2'
DECLARE @payload NVARCHAR(max) = N'{"model":"' + @model + '","prompt":"' + @prompt + '","stream": false }'

EXEC Sp_invoke_external_rest_endpoint
  @url = N'https://localhost:443/api/generate',
  @payload = @payload,
  @timeout = 230,
  @response = @response output;

SELECT value
FROM   Openjson(Json_query(@response, '$.result')) A
WHERE  [key] = 'response' 

In [ ]:
DECLARE @prompt NVARCHAR(max)

SELECT TOP 1 @prompt = 'You are at a conference called PASS Data Community Summit and attended the session below. Please write a Linkedin post about what you liked about the session. '
          + mainspeaker + ' -' + title + ' -' + [description]
FROM   conferencesessions.dbo.sessions
WHERE  speakers LIKE '%weissman%'

SELECT @prompt 

In [ ]:
DECLARE @prompt NVARCHAR(max)

SELECT TOP 1 @prompt = 'You are at a conference called PASS Data Community Summit and attended the session below. Please write a Linkedin post about what you liked about the session. '
          + mainspeaker + ' -' + title + ' -' + [description]
FROM   conferencesessions.dbo.sessions
WHERE  speakers LIKE '%weissman%'

SET @prompt = Replace(Replace(@prompt, Char(13), ''), Char(10), '')

DECLARE @response NVARCHAR(max)
DECLARE @model NVARCHAR(250) = N'smollm2'
DECLARE @payload NVARCHAR(max) = N'{"model":"' + @model + '","prompt":"' + @prompt + '","stream": false }'

EXEC Sp_invoke_external_rest_endpoint
  @url = N'https://localhost:443/api/generate',
  @payload = @payload,
  @timeout = 230,
  @response = @response output;

SELECT value FROM   Openjson(Json_query(@response, '$.result')) A WHERE  [key] = 'response' 

In [ ]:
DECLARE @apiKey NVARCHAR(255) = 'XXX'
DECLARE @url NVARCHAR(max) =  N'https://api.exchangeratesapi.io/v1/latest?access_key=' + @apikey + '&base=EUR&symbols=USD,GBP'
DECLARE @response NVARCHAR(max)

EXEC Sp_invoke_external_rest_endpoint
  @url = @url,
  @response = @response output

SELECT [key],
       value
FROM   Openjson(Json_query(@response, '$.result.rates'), 'strict $') AS export 

In [ ]:
CREATE DATABASE SCOPED CREDENTIAL [https://api.exchangeratesapi.io/]
WITH IDENTITY = 'HTTPEndpointHeaders', SECRET = '{"access_key":"XXX"}';

In [ ]:
DECLARE @url NVARCHAR(max) = N'https://api.exchangeratesapi.io/v1/latest?base=EUR&symbols=USD,GBP'
DECLARE @response NVARCHAR(max)

EXEC Sp_invoke_external_rest_endpoint
  @credential = [https://api.exchangeratesapi.io/],
  @url = @url,
  @response = @response output

SELECT [key],
       value
FROM   Openjson(Json_query(@response, '$.result'), 'strict $') AS export 

In [ ]:
ALTER DATABASE SCOPED CREDENTIAL [https://api.exchangeratesapi.io/]
WITH IDENTITY = 'HttpEndpointQueryString', SECRET = '{"access_key":"XXX"}';

In [ ]:
DECLARE @url NVARCHAR(max) = N'https://api.exchangeratesapi.io/v1/latest?base=EUR&symbols=USD,GBP'
DECLARE @response NVARCHAR(max)

EXEC Sp_invoke_external_rest_endpoint
  @credential = [https://api.exchangeratesapi.io/],
  @url = @url,
  @response = @response output

SELECT [key],
       value
FROM   Openjson(Json_query(@response, '$.result.rates'), 'strict $') AS export 

In [ ]:
DECLARE @Result int
DECLARE @url NVARCHAR(max) = N'https://api.exchangeratesapi.io/v1/latest?base=EUR&symbols=USD,GBP'
DECLARE @response NVARCHAR(max)

EXEC @Result =  Sp_invoke_external_rest_endpoint
  @credential = [https://api.exchangeratesapi.io/],
  @url = @url,
  @response = @response output

SELECT @result,@response

In [ ]:
DECLARE @Result int
DECLARE @url NVARCHAR(max) = N'https://api.exchangeratesapi.io/v1/latest?base=EUR&symbols=USD,GBP'
DECLARE @response NVARCHAR(max)

EXEC @Result =  Sp_invoke_external_rest_endpoint
  @url = @url,
  @response = @response output

SELECT @result,@response

In [ ]:
IF EXISTS (SELECT * FROM sys.database_scoped_credentials WHERE NAME = 'https://test.api.amadeus.com/')
  BEGIN
      DROP DATABASE scoped credential [https://test.api.amadeus.com/];
  END

GO

DECLARE @response NVARCHAR(max)

EXEC Sp_invoke_external_rest_endpoint
  @url = N'https://test.api.amadeus.com/v1/security/oauth2/token',
  @headers='{"content-type":"application/x-www-form-urlencoded"}',
  @payload=N'grant_type=client_credentials&client_id=XXX&client_secret=XXX',
  @method = 'POST',
  @response = @response output;

DECLARE @secret NVARCHAR(max)

SELECT @secret = value
FROM   Openjson(Json_query(@response, '$.result')) A
WHERE  [key] = 'access_token'

SET @secret = N'{"Authorization":"Bearer ' + @secret + '"}'
SET @secret = N'CREATE DATABASE SCOPED CREDENTIAL [https://test.api.amadeus.com/] WITH IDENTITY = ''HTTPEndpointHeaders'', SECRET = ''' + @secret + ''''

SELECT @secret

EXEC Sp_executesql  @secret 

In [ ]:
DECLARE @response NVARCHAR(max)
DECLARE @From NVARCHAR(3) = 'NUE'
DECLARE @To NVARCHAR(3) = 'MEM'
DECLARE @Date NVARCHAR(10) = CONVERT(NVARCHAR, Dateadd(d, 1, Getdate()), 23)
DECLARE @url NVARCHAR(max) = N'https://test.api.amadeus.com/v2/shopping/flight-offers?originLocationCode=' + @From + '&destinationLocationCode=' + @To + '&departureDate=' + @Date + '&adults=1&travelClass=BUSINESS&includedAirlineCodes=LH,UA'

EXEC Sp_invoke_external_rest_endpoint
  @url = @url,
  @credential = [https://test.api.amadeus.com/],
  @method = 'GET',
  @response = @response output;

SELECT Json_value(value, '$.price.grandTotal')                                                     Price,
       Replace(Json_value(value, '$.itineraries[0].duration'), 'PT', '')                           Duration,
       Replace(Json_query(value, '$.itineraries[0].segments[*].arrival.iataCode'), ','+ @To, '')   Stops
FROM   Openjson(Json_query(@response, '$.result.data'))
ORDER  BY duration 

In [ ]:
IF EXISTS (SELECT * FROM sys.database_scoped_credentials WHERE NAME = 'https://management.azure.com/')
  BEGIN
      DROP DATABASE scoped credential [https://management.azure.com/];
  END

GO

DECLARE @response NVARCHAR(max)

EXEC Sp_invoke_external_rest_endpoint
  @url = N'https://login.microsoftonline.com/XXX/oauth2/token',
  @headers='{"content-type":"application/x-www-form-urlencoded"}',
  @payload=N'grant_type=client_credentials&client_id=XXX&client_secret=XXX&resource=https%3A%2F%2Fmanagement.azure.com%2F',
  @method = 'POST',
  @response = @response output;

DECLARE @secret NVARCHAR(max)

SELECT @secret = value
FROM   Openjson(Json_query(@response, '$.result')) A
WHERE  [key] = 'access_token'

SET @secret = N'{"Authorization":"Bearer ' + @secret + '"}'
SET @secret = N'CREATE DATABASE SCOPED CREDENTIAL [https://management.azure.com/] WITH IDENTITY = ''HTTPEndpointHeaders'', SECRET = ''' + @secret + ''''

EXEC Sp_executesql  @secret 

In [ ]:
DECLARE @response NVARCHAR(max)

EXEC Sp_invoke_external_rest_endpoint @url = 'https://management.azure.com/subscriptions/XXX/providers/Microsoft.Compute/disks?api-version=2025-01-02',
@credential = [https://management.azure.com/],
@method = 'GET',
@response = @response output;

SELECT *
FROM   (SELECT Json_value(value, '$.name')            DiskName,
               Json_value(value, '$.properties.tier') DiskTier
        FROM   Openjson(Json_query(@response, '$.result.value'))) A
WHERE  diskname LIKE 'SQL2025%' 